In [155]:
import torch
from torch.autograd import Variable
import warnings
from torch import nn
from collections import OrderedDict
import os
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import data as data
from data.BehavioralDataset import BehavioralDataset
from data.BehavioralHmSamples import BehavioralHmSamples
import scipy
from sklearn.preprocessing import MinMaxScaler

In [2]:
warnings.filterwarnings("ignore")

In [3]:
def load_netG(path, isize, nz, nc, ngf, n_extra_layers):
    assert isize % 16 == 0, "isize has to be a multiple of 16"

    cngf, tisize = ngf//2, 4
    while tisize != isize:
        cngf = cngf * 2
        tisize = tisize * 2

    main = nn.Sequential()
    # input is Z, going into a convolution
    main.add_module('initial:{0}-{1}:convt'.format(nz, cngf),
                    nn.ConvTranspose2d(nz, cngf, 4, 1, 0, bias=False))
    main.add_module('initial:{0}:batchnorm'.format(cngf),
                    nn.BatchNorm2d(cngf))
    main.add_module('initial:{0}:relu'.format(cngf),
                    nn.ReLU(True))

    csize, cndf = 4, cngf
    while csize < isize//2:
        main.add_module('pyramid:{0}-{1}:convt'.format(cngf, cngf//2),
                        nn.ConvTranspose2d(cngf, cngf//2, 4, 2, 1, bias=False))
        main.add_module('pyramid:{0}:batchnorm'.format(cngf//2),
                        nn.BatchNorm2d(cngf//2))
        main.add_module('pyramid:{0}:relu'.format(cngf//2),
                        nn.ReLU(True))
        cngf = cngf // 2
        csize = csize * 2

    # Extra layers
    for t in range(n_extra_layers):
        main.add_module('extra-layers-{0}:{1}:conv'.format(t, cngf),
                        nn.Conv2d(cngf, cngf, 3, 1, 1, bias=False))
        main.add_module('extra-layers-{0}:{1}:batchnorm'.format(t, cngf),
                        nn.BatchNorm2d(cngf))
        main.add_module('extra-layers-{0}:{1}:relu'.format(t, cngf),
                        nn.ReLU(True))

    main.add_module('final:{0}-{1}:convt'.format(cngf, nc),
                    nn.ConvTranspose2d(cngf, nc, 4, 2, 1, bias=False))
    main.add_module('final:{0}:tanh'.format(nc),
                    nn.Tanh())
    
    state_dict = torch.load(path, map_location=torch.device('cpu'))

    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[5:] # remove `main.`
        new_state_dict[name] = v
    
    main.load_state_dict(new_state_dict, strict=False)
    
    return main

In [4]:
def load_netG_mlp(path, isize, nz, nc, ngf):
    
    main = nn.Sequential(
        # Z goes into a linear of size: ngf
        nn.Linear(nz, ngf),
        nn.ReLU(True),
        nn.Linear(ngf, ngf),
        nn.ReLU(True),
        nn.Linear(ngf, ngf),
        nn.ReLU(True),
        nn.Linear(ngf, nc * isize * isize),
    )
    
    state_dict = torch.load(path, map_location=torch.device('cpu'))

    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[5:] # remove `main.`
        new_state_dict[name] = v
    
    main.load_state_dict(new_state_dict, strict=False)
    
    return main

In [5]:

def load_netD(path, isize, nc, ndf, n_extra_layers):
    
    assert isize % 16 == 0, "isize has to be a multiple of 16"

    main = nn.Sequential()
    # input is nc x isize x isize
    main.add_module('initial:{0}-{1}:conv'.format(nc, ndf),
                    nn.Conv2d(nc, ndf, 4, 2, 1, bias=False))
    main.add_module('initial:{0}:relu'.format(ndf),
                    nn.LeakyReLU(0.2, inplace=True))
    csize, cndf = isize / 2, ndf

    # Extra layers
    for t in range(n_extra_layers):
        main.add_module('extra-layers-{0}:{1}:conv'.format(t, cndf),
                        nn.Conv2d(cndf, cndf, 3, 1, 1, bias=False))
        main.add_module('extra-layers-{0}:{1}:batchnorm'.format(t, cndf),
                        nn.BatchNorm2d(cndf))
        main.add_module('extra-layers-{0}:{1}:relu'.format(t, cndf),
                        nn.LeakyReLU(0.2, inplace=True))

    while csize > 4:
        in_feat = cndf
        out_feat = cndf * 2
        main.add_module('pyramid:{0}-{1}:conv'.format(in_feat, out_feat),
                        nn.Conv2d(in_feat, out_feat, 4, 2, 1, bias=False))
        main.add_module('pyramid:{0}:batchnorm'.format(out_feat),
                        nn.BatchNorm2d(out_feat))
        main.add_module('pyramid:{0}:relu'.format(out_feat),
                        nn.LeakyReLU(0.2, inplace=True))
        cndf = cndf * 2
        csize = csize / 2

    # state size. K x 4 x 4
    main.add_module('final:{0}-{1}:conv'.format(cndf, 1),
                    nn.Conv2d(cndf, 1, 4, 1, 0, bias=False))

    state_dict = torch.load(path, map_location=torch.device('cpu'))

    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[5:] # remove `module.`
        new_state_dict[name] = v

    main.load_state_dict(new_state_dict, strict=False)
    return main
    

In [6]:
def load_netD_mlp(path, isize, nc, ndf):
    
    main = nn.Sequential(
            # Z goes into a linear of size: ndf
            nn.Linear(nc * isize * isize, ndf),
            nn.ReLU(True),
            nn.Linear(ndf, ndf),
            nn.ReLU(True),
            nn.Linear(ndf, ndf),
            nn.ReLU(True),
            nn.Linear(ndf, 1),
        )
    
    state_dict = torch.load(path, map_location=torch.device('cpu'))
    
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[5:] # remove `module.`
        new_state_dict[name] = v
    
    main.load_state_dict(new_state_dict, strict=False)
    return main

In [7]:
def reshape_mlp_input(input_sample):
    return input_sample.view(input_sample.size(0), 
                             input_sample.size(1) * input_sample.size(2) * input_sample.size(3))

In [25]:
def sample_wrapper(samples):
    input = torch.FloatTensor(samples.shape[0], 1, 32, 32)
    input.resize_as_(samples).copy_(samples)
    return Variable(input)

In [38]:
# load in all needed data
# load real samples
dataset_cv = BehavioralDataset(isCnnData=True, isScoring=True)
dataloader = torch.utils.data.DataLoader(dataset_cv, batch_size=3, shuffle=True, num_workers=1)
data_iter = iter(dataloader)
data = data_iter.next()
real_samples, _ = data

# load fake samples
fake_samples_list = []
for i in range(1,6):
    dataset = BehavioralHmSamples(modelNum=i, isCnnData=True, isScoring=True)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1000, shuffle=True, num_workers=1)
    data_iter = iter(dataloader)
    data = data_iter.next()
    next_samples, _ = data
    fake_samples_list.append(next_samples)

In [27]:
# read in discriminators 
num_nets = 30
netD_list = [load_netD('./loss_curves/netD_50k_{0}_automated.pth'.format(i), isize=32, nc=1, ndf=64, n_extra_layers=0) for i in range(num_nets)]

In [109]:
# score hierarchical models 
num_h_models = 5
scores = np.zeros((num_h_models, num_nets))

for j, netD in enumerate(netD_list):
    
    # score real samples 
    real_samples_scores = netD(sample_wrapper(real_samples)).data.numpy()
    
    for i, fake_samples in enumerate(fake_samples_list):
        
        fake_samples_scores = netD(sample_wrapper(fake_samples)).data.numpy()
        
        # see how many fake samples were scored as more real than each real samples
        num_right = sum([np.sum(fake_samples_scores < real_samples_scores[k]) for k in range(real_samples_scores.shape[0])])
#         print('index: ({0}, {1})'.format(i,j))
#         print(num_right)
#         print(np.sum(fake_samples_scores < real_samples_scores[0]))
#         print(np.sum(fake_samples_scores < real_samples_scores[1]))
#         print(np.sum(fake_samples_scores < real_samples_scores[2]))
#         print(num_right / (1000 * real_samples_scores.shape[0]))
#         print('-----------------------')
        scores[i,j] = num_right / (1000 * real_samples_scores.shape[0])
        

In [154]:
scores_df = pd.DataFrame(scores)
scores_df.to_csv('./data/behavioral_model_scores.csv', index=False, float_format='%.3f')

In [146]:
means = list(np.mean(scores, axis=1))
samp_se_list = list(scipy.stats.sem(scores, axis=1))
var_list = list(np.var(scores, axis=1))

Where $\hat{p}$ is the probability of a dog getting shocked.

$X_{1it}$ and $X_{2it}$ are the number of times a dog has respectively been shocked and avoided being shocked in previous trials.

Model 1 
\begin{align*}
\hat{p}=\sigma(\beta_{1}+\beta_{2}X_{1it}+\beta_{3}X_{2it})
\end{align*}

Model 2 
\begin{align*}
\hat{p}=exp(\beta_{1}X_{1it}+\beta_{2}X_{2it})
\end{align*}

Model 3 
\begin{align*}
\hat{p}&=\sigma(\frac{\alpha}{t}+\gamma)&\text{$t$ is the trial number}
\end{align*}

Model 4 is the "switch." 

Model 5 
\begin{align*}
\hat{p}&=\frac{\alpha}{t}&\text{$t$ is the trial number}
\end{align*}


In [147]:
for mean, var, se, i in zip(means, var_list, samp_se_list, range(1,len(means)+1)):
    print('Model\t {0} \nMean\t {1}\nSE\t {2}\nVar\t {3}'.format(i, mean, se, var))
    print('------------------')


Model	 1 
Mean	 0.667177777777778
SE	 0.024067406051692514
Var	 0.016797960987654324
------------------
Model	 2 
Mean	 0.7062666666666666
SE	 0.021775332570521916
Var	 0.013750788148148148
------------------
Model	 3 
Mean	 0.5994777777777779
SE	 0.023751027044182103
Var	 0.01635922728395062
------------------
Model	 4 
Mean	 0.7566777777777778
SE	 0.03867475633216149
Var	 0.043376366543209886
------------------
Model	 5 
Mean	 0.5840888888888888
SE	 0.024412917321396405
Var	 0.017283725432098763
------------------


# Scaled Scores

In [159]:
scaled_scores = scores.copy()
scaler = MinMaxScaler()
scaled_scores = scaler.fit_transform(scaled_scores)
scaled_scores_df = pd.DataFrame(scaled_scores)
scaled_scores_df.to_csv('./data/behavioral_model_scaled_scores.csv', index=False, float_format='%.3f')

In [161]:
scaled_means = list(np.mean(scaled_scores, axis=1))
scaled_samp_se_list = list(scipy.stats.sem(scaled_scores, axis=1))
scaled_var_list = list(np.var(scaled_scores, axis=1))

In [162]:
for mean, var, se, i in zip(scaled_means, scaled_var_list, scaled_samp_se_list, range(1,len(scaled_means)+1)):
    print('Model\t {0} \nMean\t {1}\nSE\t {2}\nVar\t {3}'.format(i, mean, se, var))
    print('------------------')

Model	 1 
Mean	 0.514460045178651
SE	 0.05277298885496625
Var	 0.08076466222790538
------------------
Model	 2 
Mean	 0.6847650441536497
SE	 0.042408250528907565
Var	 0.05215533167475507
------------------
Model	 3 
Mean	 0.2499346024572722
SE	 0.04968043018215691
Var	 0.07157620914944088
------------------
Model	 4 
Mean	 0.8158547428141666
SE	 0.06065434855074908
Var	 0.10668954994535694
------------------
Model	 5 
Mean	 0.28147390527489147
SE	 0.06812138886255541
Var	 0.13457518499634114
------------------
